# Stats700-002 Homework 1, Fall 2017
### Zhen Qin
#### qinzhen@umich.edu

I discussed problem 1 and 2 with Zhichao Yang and Lei Zhang. I did not discuss problem 3 with anyone.

Problem 1 took me one hour; Problem 2 took me four hours; Problem 3 took me two hours.

### Problem 1: Regular expressions

1.

Use urllib2 to read the wordlist hosted on my website and produce a list of words, store the data as **words**.

In [1]:
# Write code in code blocks.
# Here, we use the urllib2 package.

import urllib2
import re
response = urllib2.urlopen('http://www-personal.umich.edu/~klevin/teaching/Fall2017/STATS700-002/lawler_wordlist.txt')
words = response.read()
words = words.split()

2.

Here are 69903 words on the list.

In [3]:
len(words)

69903

3.

Filter the parentheses and slashes out of the list, store the new data as **word**, and I am not introducing duplicates into your filtered list.

In [3]:
regex = re.compile(r'.*/')
word = [v for v in words if not bool(re.match(regex, v))]
regex = re.compile('(.*)\(.*\)(.*)')
for i in range(len(word)):
    m = re.search(regex, word[i])
    if bool(m):
        word[i] = m.group(1) + m.group(2)
word = list(set(word))
len(word) == len(set(word))

True

4.

69408 words are left after performing the filtering in the previous step.

In [22]:
len(word)

69408

5.

990 words start with exactly two consecutive vowels.

In [5]:
regex = re.compile(r'^[aeiou]{2}[^aeiou]|^[aeiou]{2}$')
twovowel = [v for v in word if bool(re.match(regex, v))]
len(twovowel)

990

6.

25174 words do not contain the letter e.

In [6]:
noe = [v for v in word if not bool(re.match(r'.*e', v))]
len(noe)

25174

7.

199 words contain at least four instances of the letter a.

In [13]:
regex = re.compile(r'.*a.*a.*a.*a.*')
foura = [v for v in word if bool(re.match(regex, v))]
len(foura)

199

8.

196 words begin and end with a vowel with only consonants in between, here I do not count those without consonants between vowels.

In [7]:
regex = re.compile(r'^[aeiou][bcdfghjklmnpqrstvwxz]+[aeiou]$')
novowelin = [v for v in word if bool(re.match(regex, v))]
len(novowelin)

196

9.

555 words start and end with the same two-letter sequence.

In [19]:
regex = re.compile(r'^([a-z]{2}).*\1$|^.{2}$|^(.)\1{2}$')
twoseq = [v for v in word if bool(re.match(regex, v))]
len(twoseq)

555

### Problem 2: Querying and parsing webpages using beautifulsoup4



1.

Give Harbin as an example. Scrape data from Wikipedia using BeautifulSoup.

In [5]:
import bs4
from bs4 import BeautifulSoup
import urllib2
import re
res = urllib2.urlopen('https://en.wikipedia.org/wiki/Harbin')
doc = res.read()
ht = BeautifulSoup(doc, 'html.parser')

Find a Wikipedia page for Harbin that includes such a climate table. Store the table as ct.

In [6]:
tables=ht.find_all('table')
reg = re.compile(r'Climate data for .*')
for t in tables:
    ths = t.find_all(['th','caption'])
    for th in ths:
        st = th.get_text()
        b = bool(re.match(reg,st))
        if b:
            ct = t
            break            

The table has 14 children. In fact, from the top to the bottom, there are 14 different pieces that are seperated by lines. Each piece is a child of the table. The table has a header, and then a data table with column names, and finally tails that indicate the source of the table.

In [7]:
n = 0
for i in ct.children:
    if isinstance(i,bs4.element.Tag):
        n = n + 1
n

14

Its children's names are shown below. Their names are tr.

In [8]:
for i in ct.children:
    if isinstance(i,bs4.element.Tag):
        print i.name,

tr tr tr tr tr tr tr tr tr tr tr tr tr tr


Its descendants' names are shown below. Their names are th, tr, td, br, sup, a and span.

In [9]:
for i in ct.descendants:
    if isinstance(i,bs4.element.Tag):
        print i.name,

tr th tr th th th th th th th th th th th th th th tr th td br td br td br td br td br td br td br td br td br td br td br td br td br tr th td br td br td br td br td br td br td br td br td br td br td br td br td br tr th td br td br td br td br td br td br td br td br td br td br td br td br td br tr th td br td br td br td br td br td br td br td br td br td br td br td br td br tr th td br td br td br td br td br td br td br td br td br td br td br td br td br tr th a td br td br td br td br td br td br td br td br td br td br td br td br td br tr th span td td td td td td td td td td td td td tr th a td td td td td td td td td td td td td tr th a td td td td td td td td td td td td td tr th a td td td td td td td td td td td td td tr td sup a tr td sup a


For more information, the structure of the table **ct** can be printed.

2.

Write a function retrieve climate table that takes as an argument a Wikipedia URL, and returns the BeautifulSoup object corresponding to the climate data table (if it exists in the page) and returns None if no such table exists on the page.

In [10]:
def retrieve_climate_table(url):
    res = urllib2.urlopen(url)
    doc = res.read()
    ht = BeautifulSoup(doc, 'html.parser')
    tables=ht.find_all('table')
    reg = re.compile(r'Climate data for .*')
    found = False
    for t in tables:
        ths = t.find_all(['th','caption'])
        for th in ths:
            st = ''.join([unicode(e) for e in th.contents])
            b = bool(re.match(reg,st))
            if b:
                ct = t
                found = True
                break
    if found:
        return ct
    else:
        return None

Test the function by pages for Los Angeles, Hyderabad and Boston

In [7]:
retrieve_climate_table('https://en.wikipedia.org/wiki/Buenos_Aires') is None

False

In [151]:
retrieve_climate_table('https://en.wikipedia.org/wiki/Hyderabad') is None

False

In [32]:
retrieve_climate_table('https://en.wikipedia.org/wiki/Boston') is None

False

3.

Write a function list climate table row names that takes as an argument a Wikipedia URL and returns a list of the row names of the climate data table, or returns None if no such table exists.

In [11]:
def list_climate_table_row_names(url):
    rect = retrieve_climate_table(url)
    if rect is None:
        return None
    rname = []
    ths =rect.find_all('th')
    for th in ths:
        if th.get('scope')=='row':
            rname.append(th.get_text())
    return rname

Test the function by pages for Hyderabad.

In [17]:
list_climate_table_row_names('https://en.wikipedia.org/wiki/Boston')

[u'Month',
 u'Record high \xb0F (\xb0C)',
 u'Mean maximum \xb0F (\xb0C)',
 u'Average high \xb0F (\xb0C)',
 u'Daily mean \xb0F (\xb0C)',
 u'Average low \xb0F (\xb0C)',
 u'Mean minimum \xb0F (\xb0C)',
 u'Record low \xb0F (\xb0C)',
 u'Average precipitation inches (mm)',
 u'Average snowfall inches (cm)',
 u'Average precipitation days (\u2265 0.01 in)',
 u'Average snowy days (\u2265 0.1 in)',
 u'Average relative humidity (%)',
 u'Mean monthly sunshine hours',
 u'Percent possible sunshine']

4.

Pseudo codes are shown below. Here I regard the row that includes column names as a part of the data table, i.e. one row of the table.

### Problem 3: Relational Databases: SQL

1.

Load the database using the Python sqlite3 package. 

In [24]:
import sqlite3
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
c = conn.cursor()

11 tables are in the database. Their names are shown below.

In [25]:
nt = c.execute('''select count(*) from sqlite_master where type="table"''')
for i in nt:
    print i[0]

11


2.

Here are the names of the tables in the database.

In [26]:
namet = c.execute('''select name from sqlite_master where type="table"''')
for i in namet:
    print i[0],

Album Artist Customer Employee Genre Invoice InvoiceLine MediaType Playlist PlaylistTrack Track


3.

List all variables in Album.

In [27]:
al = c.execute('''Select * From Album''')
al.description

(('AlbumId', None, None, None, None, None, None),
 ('Title', None, None, None, None, None, None),
 ('ArtistId', None, None, None, None, None, None))

Write a function list album ids by letter that takes as an argument a single character and returns a list of the primary keys of all the albums whose titles start with that character.

In [28]:
def list_album_ids_by_letter(x):
    if type(x) != str or len(x) != 1:
        return None
    t = (x + '%',)
    pk = c.execute('SELECT AlbumId FROM Album WHERE Title LIKE ?', t)
    pkl = [i[0] for i in pk]
    return pkl

The function should ignore case.

In [29]:
list_album_ids_by_letter('a') == list_album_ids_by_letter('A')

True

35 albums are there in the database with titles starting with “B”.

In [30]:
len(list_album_ids_by_letter('B'))

35

10 albums are there in the database with titles starting with “H”.

In [31]:
len(list_album_ids_by_letter('H'))

10

4.

List all variables in PlaylistTrack and Track.

In [32]:
plt = c.execute('''Select * From PlaylistTrack''')
plt.description

(('PlaylistId', None, None, None, None, None, None),
 ('TrackId', None, None, None, None, None, None))

In [33]:
tr = c.execute('''Select * From Track''')
tr.description

(('TrackId', None, None, None, None, None, None),
 ('Name', None, None, None, None, None, None),
 ('AlbumId', None, None, None, None, None, None),
 ('MediaTypeId', None, None, None, None, None, None),
 ('GenreId', None, None, None, None, None, None),
 ('Composer', None, None, None, None, None, None),
 ('Milliseconds', None, None, None, None, None, None),
 ('Bytes', None, None, None, None, None, None),
 ('UnitPrice', None, None, None, None, None, None))

Write a function list song ids by album letter that takes as an argument a single character and returns a list of the primary keys of all the songs whose album names
begin with that letter (again ignoring case).

In [34]:
def list_song_ids_by_album_letter(x):
    if type(x) != str or len(x) != 1:
        return None
    t = (x + '%',)
    pk = c.execute('Select TrackId From Track JOIN Album \
                   ON Track.AlbumId = Album.AlbumId WHERE Album.Title LIKE ?', t)
    pkl = [i[0] for i in pk]
    return pkl

252 songs are in all the albums whose titles begin with the letter “m”.

In [35]:
len(list_song_ids_by_album_letter('m'))

252

0 songs are in all the albums whose titles begin with the letter “y”.

In [36]:
len(list_song_ids_by_album_letter('y'))

0

5.

Write a function total cost by album letter that takes as an argument a single character and returns the cost of buying every song whose album begins with that letter.

In [37]:
def total_cost_by_album_letter(x):
    if type(x) != str or len(x) != 1:
        return None
    t = (x + '%',)
    pk = c.execute('Select SUM(UnitPrice) From Track JOIN Album \
                   ON Track.AlbumId = Album.AlbumId WHERE Album.Title LIKE ?', t)
    pr = [i[0] for i in pk]
    if pr[0] is None:
        return 0
    else:
        return round(pr[0], 2)

It would cost to buy all tracks in the database whose albums begin with “k”.

In [42]:
total_cost_by_album_letter('k')

46.53